In [1]:
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [16]:
## Load the ANN trained model, scaler pickle file and ohe pickle file

model=load_model('model.h5')

## Load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


In [30]:
## Example input data
input_data = {
    'CreditScore': 800,
    'Geography': 'Spain',
    'Gender': 'Female',
    'Age': 40,
    'Tenure': 3,
    'Balance': 6000000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 500000
}

In [31]:
#one hot encode geography

geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

a:\ML-Project\ANN-Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [32]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,Spain,Female,40,3,6000000,2,1,1,500000


In [33]:
## Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,Spain,0,40,3,6000000,2,1,1,500000


In [34]:
## concatination with ohe data
input_df=pd.concat([input_df.drop("Geography",axis=1), geo_encoded_df],axis=1)

In [35]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,0,40,3,6000000,2,1,1,500000,0.0,0.0,1.0


In [36]:
## Scaling the data
input_scaled = scaler.transform(input_df)
input_scaled

array([[ 1.53956105, -1.09499335,  0.10479359, -0.69539349, 94.8474663 ,
         0.80843615,  0.64920267,  0.97481699,  6.94716943, -0.99850112,
        -0.57946723,  1.73494238]])

In [37]:
## predict model
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 32ms/step


array([[0.13380887]], dtype=float32)

In [38]:
prediction_proba = prediction[0][0]
prediction_proba

0.13380887

In [39]:
if prediction_proba > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
